In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 133872 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.18-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [1]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [2]:
# 코랩환경에서 작업여부를 나타내는 변수
IS_COLAB = True 

if IS_COLAB:
  path = "drive/colab/dacon_jeju_traffic/data/"
else:
  path = "I don't know yet"

print(f"path of data files is {path}")

path of data files is drive/colab/dacon_jeju_traffic/data/


In [3]:
!apt install chromium-chromedriver
!pip install selenium

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (80.0.3987.87-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [4]:
ls -l drive/colab/dacon_jeju_traffic/data

total 381773
-r--r--r-- 1 root root 289504974 Nov  7 06:53  bus_bts.csv
-rw-r--r-- 1 root root   2310165 May 11  2019  jeju_financial_life_data.csv
-rw-r--r-- 1 root root   5982011 Mar 21 06:03 'submission_데이콘 서포터즈 1기_권영인_1주차.csv'
-rw-r--r-- 1 root root   4103729 Mar  8 12:17 'submission_데이콘 서포터즈 1기_권영인.csv'
-r--r--r-- 1 root root   2281715 Nov  7 06:39  submission_sample.csv
-r--r--r-- 1 root root  29758470 Nov  7 06:40  test.csv
-r--r--r-- 1 root root  55756093 Nov  7 06:40  train.csv
-rw-r--r-- 1 root root      1372 Mar 22 08:33  weather.csv
-r--r--r-- 1 root root   1234237 Dec 21 07:47 '행정_법정동 중심좌표.xlsx'


# [데이콘 서포터즈 1기_권영인] How much heck the bus is in jeju than seoul_2nd week

## 2주차 달성 목표

1. 전반적인 우승자 코드의 clone coding
2. statsmodel.api 와 VIF를 활용한 다중공선성 문제 해결하기
3. PCA(주성분 분석) 알아보기

### Preparing Data

In [5]:
# 주요 패키지 import

import os 
import warnings
import pandas as pd
import numpy as np
import re
import datetime

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px

import folium # 지도 관련 시각화
from folium.plugins import MarkerCluster #지도 관련 시각화
import geopy.distance #거리 계산해주는 패키지 사용

from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# base
# import os
import shutil
import random
# import numpy as np
import time
from collections import Counter
from sklearn.metrics import mean_squared_error

# data preprocessing
import pandas as pd
pd.set_option('max_columns', 130, 'max_rows', 30)

# visualization
# import seaborn as sns
# import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

# ignore warining
# import warnings
# warnings.filterwarnings('ignore')


import folium # 지도 관련 시각화
from folium.plugins import MarkerCluster #지도 관련 시각화
import geopy.distance #거리 계산해주는 패키지 사용


# save
from sklearn.externals import joblib 
import pickle

# selenium

# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

from selenium.webdriver import Chrome

import geopy.distance

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import RandomForestRegressor

# import seaborn as sns
# import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals import joblib 
import pickle

import random
random.seed(42)

import lightgbm as lgb
from keras import metrics
from sklearn.metrics import r2_score

# 경고메시지 제거
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.

Using TensorFlow backend.


#### weather crwaling

In [0]:
def crawl_weather() :
    
  #https://gist.github.com/korakot/5c8e21a5af63966d80a676af0ce15067 [Colab 환경에서 selenium을 활용한 Crawling]

  # https://wkdtjsgur100.github.io/selenium-xpath/ [Selenium을 활용한 하위객체 찾기]
  weather_data_10 = pd.DataFrame(columns=['현재일기_10','현재기온_10','일강수_10'])
  browser = webdriver.Chrome('chromedriver',options=options)
  # browser = Chrome()
  url = 'http://www.weather.go.kr/weather/observation/currentweather.jsp?auto_man=m&type=t99&reg=184&tm=2019.10.25.16%3A00&x=19&y=7'
  browser.get(url)

  for i in range(0,46):
      i+=1

      elem=browser.find_element_by_id('observation_text')
      elem.clear()
      elem.send_keys("2019.9.{}.10:00".format(i))

      btn=browser.find_elements_by_class_name('btn')
      btn[2].click()
      
      time.sleep(1)
      weathers = browser.find_elements_by_css_selector('tr')

      mean_temp_now = 0
      mean_rain_amount = 0

      for tr in weathers[2:]:
        tds = tr.find_elements_by_tag_name("td")

        if tds[0].find_element_by_tag_name("a").text == "제주":
          weather = tds[1].text

        mean_temp_now+=float(tds[5].text)
        if tds[8].text == " ":
          mean_rain_amount += 0.0
        else:
          mean_rain_amount += float(tds[8].text)

      mean_temp_now = mean_temp_now / len(weathers[2:])
      mean_rain_amount = mean_rain_amount / len(weathers[2:])

      weather_data_10  = weather_data_10.append(pd.DataFrame([[weather,mean_temp_now,mean_rain_amount]],columns=['현재일기_10','현재기온_10','일강수_10'])) 

  print('success !')
  browser.close()
  
  return weather_data_10

In [0]:
# weather_data = crawl_weather()

In [0]:
# weather_data.to_csv(path+'weather.csv', index = False)

In [0]:
weather_data = pd.read_csv(path+'weather.csv')

In [0]:
train = pd.read_csv(path+"train.csv")
test = pd.read_csv(path+"test.csv")
bts = pd.read_csv(path+"bus_bts.csv")

### EDA

#### 버스 승하차 데이터

##### day, day_type[weekday | weekend] & day_name(Mon to Sun)

In [0]:
day_type_dict = {0:"weekday",1:"weekday",2:"weekday",3:"weekday",4:"weekday",5:"weekend",6:"weekend"}
day_name_dict = {0:"Mon",1:"Tue",2:"Wed",3:"Thr",4:"Fri",5:"Sat",6:"Sun"}

train_date = train.date.value_counts().to_frame().sort_index().reset_index()
train_date.columns = ["date","count"]
train_date["month"] = "Sep"
train_date["day"] = train_date.date.apply(lambda x: x.split("-")[2])


test_date = test.date.value_counts().to_frame().sort_index().reset_index()
test_date.columns = ["date","count"]
test_date["month"] = "Oct"
test_date["day"] = test_date.date.apply(lambda x: x.split("-")[2])

# df_train_test_date = pd.concat([df_train_date,df_test_date],axis=0)
train_date["day_type"] = train_date.date.apply(lambda x: day_type_dict[datetime.datetime.strptime(x,"%Y-%m-%d").weekday()])
train_date["day_name"] = train_date.date.apply(lambda x: day_name_dict[datetime.datetime.strptime(x,"%Y-%m-%d").weekday()])

test_date["day_type"] = test_date.date.apply(lambda x: day_type_dict[datetime.datetime.strptime(x,"%Y-%m-%d").weekday()])
test_date["day_name"] = test_date.date.apply(lambda x: day_name_dict[datetime.datetime.strptime(x,"%Y-%m-%d").weekday()])

In [0]:
train = pd.merge(left=train,right=train_date.loc[:,["date","day","day_type","day_name"]],how='left',on='date')
test = pd.merge(left=test,right=test_date.loc[:,["date","day","day_type","day_name"]],how='left',on='date')

In [0]:
for df in [train,test]:
  df["is_weekend"] = df["day_type"].apply(lambda x: True if x=="weekend" else False)

In [0]:
encoder = OneHotEncoder()
encoded_day_type = encoder.fit_transform(train["day_name"].values.reshape(-1,1))
dummy = pd.DataFrame(encoded_day_type.todense(),columns=map(lambda x:x[3:] ,encoder.get_feature_names()))
dummy = dummy.loc[:,dummy.columns[:-1]]
train = pd.concat([train,dummy],axis=1)

encoded_day_type = encoder.transform(test["day_name"].values.reshape(-1,1))
dummy = pd.DataFrame(encoded_day_type.todense(),columns=map(lambda x:x[3:] ,encoder.get_feature_names()))
dummy = dummy.loc[:,dummy.columns[:-1]]
test = pd.concat([test,dummy],axis=1)

##### 공휴일

In [0]:
hoillday_date_list = ["2019-09-12","2019-09-13","2019-09-14","2019-10-03","2019-10-09"]

def filter_hoilday(strdate,list):

  if strdate in list:
    return True
  else:
    return False

In [0]:
train["is_holiday"] =train.date.map(lambda x: filter_hoilday(x,hoillday_date_list))
test["is_holiday"] =test.date.map(lambda x: filter_hoilday(x,hoillday_date_list))

##### 요일 및 평일/주말에 따른 시간대별 총 승하차 인원 수

In [0]:
geton_timeline_list = train.columns[8:14]
takeoff_timeline_list = train.columns[14:20]

In [0]:
def day_name_type_sum (df):

  # 요일별 승차 합
  f = df.groupby("day_name")[geton_timeline_list].sum(axis=0).reset_index()
  f.columns = [f.columns[0]] + list(map(lambda x:"day_name_"+x+"_sum",f.columns[1:]))
  df = pd.merge(left=df,right=f,how="left",on="day_name")

  # 요일별 승차 평균
  f = df.groupby("day_name")[geton_timeline_list].agg("mean").reset_index()
  f.columns = [f.columns[0]] + list(map(lambda x:"day_name_"+x+"_mean",f.columns[1:]))
  df = pd.merge(left=df,right=f,how="left",on="day_name")

  # 요일별 하차 합
  f = df.groupby("day_name")[takeoff_timeline_list].sum().reset_index()
  f.columns = [f.columns[0]] + list(map(lambda x:"day_name_"+x+"_sum",f.columns[1:]))
  df = pd.merge(left=df,right=f,how="left")

  # 요일별 하차 평균
  f = df.groupby("day_name")[takeoff_timeline_list].agg("mean").reset_index()
  f.columns = [f.columns[0]] + list(map(lambda x:"day_name_"+x+"_mean",f.columns[1:]))
  df = pd.merge(left=df,right=f,how="left")

  # 평/공휴일 별 승차 합
  f = df.groupby("day_type")[geton_timeline_list].sum(axis=0).reset_index()
  f.columns = [f.columns[0]] + list(map(lambda x:"day_type_"+x+"_sum",f.columns[1:]))
  df = pd.merge(left=df,right=f,how="left")

  # 평/공휴일 별 승차 평균
  f = df.groupby("day_type")[geton_timeline_list].agg("mean").reset_index()
  f.columns = [f.columns[0]] + list(map(lambda x:"day_type_"+x+"_mean",f.columns[1:]))
  df = pd.merge(left=df,right=f,how="left")

  # 평/공휴일 별 하차 합
  f = df.groupby("day_type")[takeoff_timeline_list].sum().reset_index()
  f.columns = [f.columns[0]] + list(map(lambda x:"day_type_"+x+"_sum",f.columns[1:]))
  df = pd.merge(left=df,right=f,how="left")

  # 평/공휴일 별 하차 평균
  f = df.groupby("day_type")[takeoff_timeline_list].agg("mean").reset_index()
  f.columns = [f.columns[0]] + list(map(lambda x:"day_type_"+x+"_mean",f.columns[1:]))
  df = pd.merge(left=df,right=f,how="left")

  return df

In [0]:
train = day_name_type_sum(train)
test = day_name_type_sum(test)

In [20]:
train.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,day,day_type,day_name,is_weekend,Fri,Mon,Sat,Sun,Thr,Tue,is_holiday,day_name_6~7_ride_sum,day_name_7~8_ride_sum,day_name_8~9_ride_sum,day_name_9~10_ride_sum,day_name_10~11_ride_sum,day_name_11~12_ride_sum,day_name_6~7_ride_mean,day_name_7~8_ride_mean,day_name_8~9_ride_mean,day_name_9~10_ride_mean,day_name_10~11_ride_mean,day_name_11~12_ride_mean,day_name_6~7_takeoff_sum,day_name_7~8_takeoff_sum,day_name_8~9_takeoff_sum,day_name_9~10_takeoff_sum,day_name_10~11_takeoff_sum,day_name_11~12_takeoff_sum,day_name_6~7_takeoff_mean,day_name_7~8_takeoff_mean,day_name_8~9_takeoff_mean,day_name_9~10_takeoff_mean,day_name_10~11_takeoff_mean,day_name_11~12_takeoff_mean,day_type_6~7_ride_sum,day_type_7~8_ride_sum,day_type_8~9_ride_sum,day_type_9~10_ride_sum,day_type_10~11_ride_sum,day_type_11~12_ride_sum,day_type_6~7_ride_mean,day_type_7~8_ride_mean,day_type_8~9_ride_mean,day_type_9~10_ride_mean,day_type_10~11_ride_mean,day_type_11~12_ride_mean,day_type_6~7_takeoff_sum,day_type_7~8_takeoff_sum,day_type_8~9_takeoff_sum,day_type_9~10_takeoff_sum,day_type_10~11_takeoff_sum,day_type_11~12_takeoff_sum,day_type_6~7_takeoff_mean,day_type_7~8_takeoff_mean,day_type_8~9_takeoff_mean,day_type_9~10_takeoff_mean,day_type_10~11_takeoff_mean,day_type_11~12_takeoff_mean
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.3767,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.21764,0.383422,0.508273,0.551915,0.56895,0.580062,9280.0,21486.0,30847.0,35133.0,36898.0,38360.0,0.094604,0.219037,0.314467,0.35816,0.376153,0.391057
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.3767,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.21764,0.383422,0.508273,0.551915,0.56895,0.580062,9280.0,21486.0,30847.0,35133.0,36898.0,38360.0,0.094604,0.219037,0.314467,0.35816,0.376153,0.391057
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.3767,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.21764,0.383422,0.508273,0.551915,0.56895,0.580062,9280.0,21486.0,30847.0,35133.0,36898.0,38360.0,0.094604,0.219037,0.314467,0.35816,0.376153,0.391057
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,6.0,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.3767,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.21764,0.383422,0.508273,0.551915,0.56895,0.580062,9280.0,21486.0,30847.0,35133.0,36898.0,38360.0,0.094604,0.219037,0.314467,0.35816,0.376153,0.391057
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.

##### 시내/시외 버스별 평균 승하차인원

In [0]:
def in_out_bus_ride_takeoff_mean_sum(df):
  
  f = df.groupby("in_out")[geton_timeline_list].mean().reset_index()
  f.columns = [f.columns[0]] + list(map(lambda x:"in_out_"+x+"_mean",f.columns[1:]))
  df = pd.merge(left=df,right=f,how="left")

  f = df.groupby("in_out")[geton_timeline_list].sum().reset_index()
  f.columns = [f.columns[0]] + list(map(lambda x:"in_out_"+x+"_sum",f.columns[1:]))
  df = pd.merge(left=df,right=f,how="left")

  f = df.groupby("in_out")[takeoff_timeline_list].mean().reset_index()
  f.columns = [f.columns[0]] + list(map(lambda x:"in_out_"+x+"_mean",f.columns[1:]))
  df = pd.merge(left=df,right=f,how="left")

  f = df.groupby("in_out")[takeoff_timeline_list].sum().reset_index()
  f.columns = [f.columns[0]] + list(map(lambda x:"in_out_"+x+"_sum",f.columns[1:]))
  df = pd.merge(left=df,right=f,how="left")

  return df

In [0]:
train = in_out_bus_ride_takeoff_mean_sum(train)
test = in_out_bus_ride_takeoff_mean_sum(test)

##### 좌표데이터를 이용한 변수

In [0]:
def geological_encoding(df):

  data = df.copy()

  jeju=(33.51411, 126.52969) # 제주 측정소 근처
  gosan=(33.29382, 126.16283) #고산 측정소 근처
  seongsan=(33.38677, 126.8802) #성산 측정소 근처
  po=(33.24616, 126.5653) #서귀포 측정소 근처
  
  print("#"*10,"jeju processing")
  t1 = [geopy.distance.vincenty( (i,j), jeju).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]
  print("#"*10,"gosan processing")
  t2 = [geopy.distance.vincenty( (i,j), gosan).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]
  print("#"*10,"seongsan processing")
  t3 = [geopy.distance.vincenty( (i,j), seongsan).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]
  print("#"*10,"seoguipo processing")
  t4 = [geopy.distance.vincenty( (i,j), po).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]

  data['dis_jeju'] = t1
  data['dis_gosan']=t2
  data['dis_seongsan']=t3
  data['dis_po']=t4

  print("#"*5,"result aggregating processing")
  total = pd.DataFrame(list(zip( t1,t2,t3,t4)),columns=['jeju','gosan','seongsan','po'] )
  data['dist_name'] = total.apply(lambda x: x.argmin(), axis=1)

  print("#"*5,"geological converting done")
    
  return data

In [24]:
train = geological_encoding(train)
test = geological_encoding(test)

########## jeju processing
########## gosan processing
########## seongsan processing
########## seoguipo processing
##### result aggregating processing
##### geological converting done
########## jeju processing
########## gosan processing
########## seongsan processing
########## seoguipo processing
##### result aggregating processing
##### geological converting done


In [25]:
train.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,day,day_type,day_name,is_weekend,Fri,Mon,Sat,Sun,Thr,Tue,is_holiday,day_name_6~7_ride_sum,day_name_7~8_ride_sum,day_name_8~9_ride_sum,day_name_9~10_ride_sum,day_name_10~11_ride_sum,day_name_11~12_ride_sum,day_name_6~7_ride_mean,day_name_7~8_ride_mean,day_name_8~9_ride_mean,day_name_9~10_ride_mean,day_name_10~11_ride_mean,day_name_11~12_ride_mean,day_name_6~7_takeoff_sum,day_name_7~8_takeoff_sum,day_name_8~9_takeoff_sum,day_name_9~10_takeoff_sum,day_name_10~11_takeoff_sum,day_name_11~12_takeoff_sum,day_name_6~7_takeoff_mean,day_name_7~8_takeoff_mean,day_name_8~9_takeoff_mean,day_name_9~10_takeoff_mean,day_name_10~11_takeoff_mean,day_name_11~12_takeoff_mean,day_type_6~7_ride_sum,day_type_7~8_ride_sum,day_type_8~9_ride_sum,day_type_9~10_ride_sum,day_type_10~11_ride_sum,day_type_11~12_ride_sum,day_type_6~7_ride_mean,day_type_7~8_ride_mean,day_type_8~9_ride_mean,day_type_9~10_ride_mean,day_type_10~11_ride_mean,day_type_11~12_ride_mean,day_type_6~7_takeoff_sum,day_type_7~8_takeoff_sum,day_type_8~9_takeoff_sum,day_type_9~10_takeoff_sum,day_type_10~11_takeoff_sum,day_type_11~12_takeoff_sum,day_type_6~7_takeoff_mean,day_type_7~8_takeoff_mean,day_type_8~9_takeoff_mean,day_type_9~10_takeoff_mean,day_type_10~11_takeoff_mean,day_type_11~12_takeoff_mean,in_out_6~7_ride_mean,in_out_7~8_ride_mean,in_out_8~9_ride_mean,in_out_9~10_ride_mean,in_out_10~11_ride_mean,in_out_11~12_ride_mean,in_out_6~7_ride_sum,in_out_7~8_ride_sum,in_out_8~9_ride_sum,in_out_9~10_ride_sum,in_out_10~11_ride_sum,in_out_11~12_ride_sum,in_out_6~7_takeoff_mean,in_out_7~8_takeoff_mean,in_out_8~9_takeoff_mean,in_out_9~10_takeoff_mean,in_out_10~11_takeoff_mean,in_out_11~12_takeoff_mean,in_out_6~7_takeoff_sum,in_out_7~8_takeoff_sum,in_out_8~9_takeoff_sum,in_out_9~10_takeoff_sum,in_out_10~11_takeoff_sum,in_out_11~12_takeoff_sum,dis_jeju,dis_gosan,dis_seongsan,dis_po,dist_name
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.3767,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.21764,0.383422,0.508273,0.551915,0.56895,0.580062,9280.0,21486.0,30847.0,35133.0,36898.0,38360.0,0.094604,0.219037,0.314467,0.35816,0.376153,0.391057,0.536617,0.997256,1.218114,1.025278,0.983100,0.946988,3715.0,6904.0,8433.0,7098.0,6806.0,6556.0,0.236458,0.530406,0.753286,0.630651,0.511195,0.474505,1637.0,3672.0,5215.0,4366.0,3539.0,3285.0,4.286659,37.692766,37.713630,27.841879,jeju
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.3767,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.21764,0.383422,0.508273,0.551915,0.56895,0.580062,9280.0,21486.0,30847.0,35133.0,36898.0,38360.0,0.094604,0.219037,0.314467,0.35816,0.376153,0.391057,0.536617,0.997256,1.218114,1.025278,0.983100,0.946988,3715.0,6904.0,8433.0,7098.0,6806.0,6556.0,0.236458,0.530406,0.753286,0.630651,0.511195,0.474505,1637.0,3672.0,5215.0,4366.0,3539.0,3285.0,4.966848,37.008492,38.465822,27.996179,jeju
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.3

In [0]:
encoder = OneHotEncoder()
encoded_day_type = encoder.fit_transform(train["dist_name"].values.reshape(-1,1))
dummy = pd.DataFrame(encoded_day_type.todense(),columns=map(lambda x:x[3:] ,encoder.get_feature_names()))
dummy = dummy.loc[:,dummy.columns[:-1]]
train = pd.concat([train,dummy],axis=1)

encoded_day_type = encoder.transform(test["dist_name"].values.reshape(-1,1))
dummy = pd.DataFrame(encoded_day_type.todense(),columns=map(lambda x:x[3:] ,encoder.get_feature_names()))
dummy = dummy.loc[:,dummy.columns[:-1]]
test = pd.concat([test,dummy],axis=1)

In [27]:
train.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,day,day_type,day_name,is_weekend,Fri,Mon,Sat,Sun,Thr,Tue,is_holiday,day_name_6~7_ride_sum,day_name_7~8_ride_sum,day_name_8~9_ride_sum,day_name_9~10_ride_sum,day_name_10~11_ride_sum,day_name_11~12_ride_sum,day_name_6~7_ride_mean,day_name_7~8_ride_mean,day_name_8~9_ride_mean,day_name_9~10_ride_mean,day_name_10~11_ride_mean,day_name_11~12_ride_mean,day_name_6~7_takeoff_sum,day_name_7~8_takeoff_sum,day_name_8~9_takeoff_sum,day_name_9~10_takeoff_sum,day_name_10~11_takeoff_sum,day_name_11~12_takeoff_sum,day_name_6~7_takeoff_mean,day_name_7~8_takeoff_mean,day_name_8~9_takeoff_mean,day_name_9~10_takeoff_mean,day_name_10~11_takeoff_mean,day_name_11~12_takeoff_mean,day_type_6~7_ride_sum,day_type_7~8_ride_sum,day_type_8~9_ride_sum,day_type_9~10_ride_sum,day_type_10~11_ride_sum,day_type_11~12_ride_sum,day_type_6~7_ride_mean,day_type_7~8_ride_mean,day_type_8~9_ride_mean,day_type_9~10_ride_mean,day_type_10~11_ride_mean,day_type_11~12_ride_mean,day_type_6~7_takeoff_sum,day_type_7~8_takeoff_sum,day_type_8~9_takeoff_sum,day_type_9~10_takeoff_sum,day_type_10~11_takeoff_sum,day_type_11~12_takeoff_sum,day_type_6~7_takeoff_mean,day_type_7~8_takeoff_mean,day_type_8~9_takeoff_mean,day_type_9~10_takeoff_mean,day_type_10~11_takeoff_mean,day_type_11~12_takeoff_mean,in_out_6~7_ride_mean,in_out_7~8_ride_mean,in_out_8~9_ride_mean,in_out_9~10_ride_mean,in_out_10~11_ride_mean,in_out_11~12_ride_mean,in_out_6~7_ride_sum,in_out_7~8_ride_sum,in_out_8~9_ride_sum,in_out_9~10_ride_sum,in_out_10~11_ride_sum,in_out_11~12_ride_sum,in_out_6~7_takeoff_mean,in_out_7~8_takeoff_mean,in_out_8~9_takeoff_mean,in_out_9~10_takeoff_mean,in_out_10~11_takeoff_mean,in_out_11~12_takeoff_mean,in_out_6~7_takeoff_sum,in_out_7~8_takeoff_sum,in_out_8~9_takeoff_sum,in_out_9~10_takeoff_sum,in_out_10~11_takeoff_sum,in_out_11~12_takeoff_sum,dis_jeju,dis_gosan,dis_seongsan,dis_po,dist_name,gosan,jeju,po
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.3767,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.21764,0.383422,0.508273,0.551915,0.56895,0.580062,9280.0,21486.0,30847.0,35133.0,36898.0,38360.0,0.094604,0.219037,0.314467,0.35816,0.376153,0.391057,0.536617,0.997256,1.218114,1.025278,0.983100,0.946988,3715.0,6904.0,8433.0,7098.0,6806.0,6556.0,0.236458,0.530406,0.753286,0.630651,0.511195,0.474505,1637.0,3672.0,5215.0,4366.0,3539.0,3285.0,4.286659,37.692766,37.713630,27.841879,jeju,0.0,1.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.3767,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.21764,0.383422,0.508273,0.551915,0.56895,0.580062,9280.0,21486.0,30847.0,35133.0,36898.0,38360.0,0.094604,0.219037,0.314467,0.35816,0.376153,0.391057,0.536617,0.997256,1.218114,1.025278,0.983100,0.946988,3715.0,6904.0,8433.0,7098.0,6806.0,6556.0,0.236458,0.530406,0.753286,0.630651,0.511195,0.474505,1637.0,3672.0,5215.0,4366.0,3539.0,3285.0,4.966848,37.008492,38.465822,27.996179,jeju,0.0,1.0,0.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,

##### 출근 시간의 총 승객 수

In [0]:
geton_timeline_list = train.columns[8:14]
takeoff_timeline_list = train.columns[14:20]

for df in [train,test]:

  df["ride_sum"] = df.loc[:,geton_timeline_list].sum(axis=1)
  df["takeoff_sum"]= df.loc[:,takeoff_timeline_list].sum(axis=1)

##### 날짜 및 시간대 별 총 승객 수

In [0]:
def geton_timeline_sum(df):
  for element in geton_timeline_list:
    f = df.groupby("date")[element].agg([(element+"_date_sum","sum")]).reset_index()
    df = pd.merge(left=df,right=f,how="left",on="date")

  return df

In [0]:
train= geton_timeline_sum(train)
test = geton_timeline_sum(test)

##### 승하차 시간대 시간단위 추가 (t~t+2)


In [0]:
for df in [train,test]:

  df["6~8r"] = df["6~7_ride"] + df["7~8_ride"]
  df["8~10r"] = df["8~9_ride"] + df["9~10_ride"]
  df["10~12r"] = df["10~11_ride"] + df["11~12_ride"]

  df["6~8t"] = df["6~7_takeoff"] + df["7~8_takeoff"]
  df["8~10t"] = df["8~9_takeoff"] + df["9~10_takeoff"]
  df["10~12t"] = df["10~11_takeoff"] + df["11~12_takeoff"]

#### Target Value를 활용한 집계성 컬럼 생성 (1)

Target Value를 통해 집계성 컬럼 생성이 가능한 이유는, 하나의 Row는 ID로 구분이 되지만 실질적인 구분자는 date,bus_route_id,station_code의 조합이기 때문이다.

In [0]:
train["route_station_code"] = train["bus_route_id"].astype("str")+"_"+train["station_code"].astype("str")
test["route_station_code"] = test["bus_route_id"].astype("str")+"_"+test["station_code"].astype("str")

In [0]:
column_list = ["bus_route_id","station_code","route_station_code"]

for element in column_list:
  agg_train= train.groupby(element)["18~20_ride"].agg([(element+"_mean","mean"),(element+"_sum","sum")]).reset_index()

  train = pd.merge(left=train,right=agg_train,how="left",on=element)
  test = pd.merge(left=test,right=agg_train,how="left",on=element)

  test[element+"_mean"] = test[element+"_mean"].fillna(agg_train[element+"_mean"].mean())
  test[element+"_sum"] = test[element+"_sum"].fillna(agg_train[element+"_sum"].mean())

#### Target Value를 활용한 집계성 컬럼 생성 (2) - 혼잡도 계산

In [0]:
congestion = train.groupby("bus_route_id")["18~20_ride"].agg([("passenger","sum")]).astype("int64").reset_index()

def congestion_level_converter(passenger):
  if passenger > 10000:
    return 7
  elif passenger > 5000:
    return 6
  elif passenger > 2000:
    return 5
  elif passenger > 700:
    return 4
  elif passenger > 200:
    return 3
  elif passenger > 50:
    return 2
  else:
    return 1

congestion["congestion"] = congestion.passenger.apply(lambda x: congestion_level_converter(x))

train = pd.merge(left=train,right=congestion.loc[:,["bus_route_id","congestion"]],how="left",on="bus_route_id")
test = pd.merge(left=test,right=congestion.loc[:,["bus_route_id","congestion"]],how="left",on="bus_route_id")

test["congestion"]= test["congestion"].fillna(train["congestion"].median())

In [35]:
train.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,day,day_type,day_name,is_weekend,Fri,Mon,Sat,Sun,Thr,Tue,is_holiday,day_name_6~7_ride_sum,day_name_7~8_ride_sum,day_name_8~9_ride_sum,day_name_9~10_ride_sum,day_name_10~11_ride_sum,day_name_11~12_ride_sum,day_name_6~7_ride_mean,day_name_7~8_ride_mean,day_name_8~9_ride_mean,day_name_9~10_ride_mean,day_name_10~11_ride_mean,day_name_11~12_ride_mean,day_name_6~7_takeoff_sum,day_name_7~8_takeoff_sum,day_name_8~9_takeoff_sum,day_name_9~10_takeoff_sum,day_name_10~11_takeoff_sum,day_name_11~12_takeoff_sum,day_name_6~7_takeoff_mean,day_name_7~8_takeoff_mean,day_name_8~9_takeoff_mean,day_name_9~10_takeoff_mean,day_name_10~11_takeoff_mean,day_name_11~12_takeoff_mean,day_type_6~7_ride_sum,day_type_7~8_ride_sum,day_type_8~9_ride_sum,day_type_9~10_ride_sum,day_type_10~11_ride_sum,day_type_11~12_ride_sum,day_type_6~7_ride_mean,day_type_7~8_ride_mean,day_type_8~9_ride_mean,...,day_type_7~8_takeoff_sum,day_type_8~9_takeoff_sum,day_type_9~10_takeoff_sum,day_type_10~11_takeoff_sum,day_type_11~12_takeoff_sum,day_type_6~7_takeoff_mean,day_type_7~8_takeoff_mean,day_type_8~9_takeoff_mean,day_type_9~10_takeoff_mean,day_type_10~11_takeoff_mean,day_type_11~12_takeoff_mean,in_out_6~7_ride_mean,in_out_7~8_ride_mean,in_out_8~9_ride_mean,in_out_9~10_ride_mean,in_out_10~11_ride_mean,in_out_11~12_ride_mean,in_out_6~7_ride_sum,in_out_7~8_ride_sum,in_out_8~9_ride_sum,in_out_9~10_ride_sum,in_out_10~11_ride_sum,in_out_11~12_ride_sum,in_out_6~7_takeoff_mean,in_out_7~8_takeoff_mean,in_out_8~9_takeoff_mean,in_out_9~10_takeoff_mean,in_out_10~11_takeoff_mean,in_out_11~12_takeoff_mean,in_out_6~7_takeoff_sum,in_out_7~8_takeoff_sum,in_out_8~9_takeoff_sum,in_out_9~10_takeoff_sum,in_out_10~11_takeoff_sum,in_out_11~12_takeoff_sum,dis_jeju,dis_gosan,dis_seongsan,dis_po,dist_name,gosan,jeju,po,ride_sum,takeoff_sum,6~7_ride_date_sum,7~8_ride_date_sum,8~9_ride_date_sum,9~10_ride_date_sum,10~11_ride_date_sum,11~12_ride_date_sum,6~8r,8~10r,10~12r,6~8t,8~10t,10~12t,route_station_code,bus_route_id_mean,bus_route_id_sum,station_code_mean,station_code_sum,route_station_code_mean,route_station_code_sum,congestion
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.3767,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.21764,0.383422,0.508273,...,21486.0,30847.0,35133.0,36898.0,38360.0,0.094604,0.219037,0.314467,0.35816,0.376153,0.391057,0.536617,0.997256,1.218114,1.025278,0.983100,0.946988,3715.0,6904.0,8433.0,7098.0,6806.0,6556.0,0.236458,0.530406,0.753286,0.630651,0.511195,0.474505,1637.0,3672.0,5215.0,4366.0,3539.0,3285.0,4.286659,37.692766,37.713630,27.841879,jeju,0.0,1.0,0.0,16.0,0.0,2942.0,4518.0,5682.0,6275.0,6774.0,6731.0,1.0,7.0,8.0,0.0,0.0,0.0,4270000_344,3.104381,2409.0,1.466667,44.0,1.466667,44.0,5
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.3767,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.21764,0.383422,0.508273,...,21486.0,30847.0,35133.0,36898.0,38360.0,0.094604,0.219037,0.314467,0.35816,0.376153,0.391057,0.536617,0.997256,1.218114,1.025278,0.983100,0.946988,3715.0,6904.0,8433.0,7098.0,6806.0,6556.0,0.236458,0.530406,0.753286,0.630651,0.511195,0.474505,1637.0,3672.0,5215.0,4366.0,3539.0,3285.0,4.966848,37.008492,38.465822,27.996179,jeju,0.0,1.0,0.0,

#### route_station_code를 활용한 전 시간대 별 집계성 컬럼 생성

In [36]:
timeline_columns = list(train.columns[8:20])
timeline_columns

['6~7_ride',
 '7~8_ride',
 '8~9_ride',
 '9~10_ride',
 '10~11_ride',
 '11~12_ride',
 '6~7_takeoff',
 '7~8_takeoff',
 '8~9_takeoff',
 '9~10_takeoff',
 '10~11_takeoff',
 '11~12_takeoff']

In [0]:
timeline_columns = list(train.columns[8:20])

for element in timeline_columns:
  agg_train =train.groupby("route_station_code")[element].agg([(element+"_mean","mean"),(element+"_sum","sum")]).reset_index()

  train = pd.merge(left=train,right=agg_train,how="left",on="route_station_code")
  test = pd.merge(left=test,right=agg_train,how="left",on="route_station_code")
  
  test[element+"_mean"] = test[element+"_mean"].fillna(agg_train[element+"_mean"].mean())
  test[element+"_sum"] = test[element+"_sum"].fillna(agg_train[element+"_sum"].mean())

#### Categorical value makes Category

In [0]:
encoder = LabelEncoder()

train["is_train"] = True
test["is_train"] = False

train_columns = train.columns

data = pd.concat([train,test],axis=0)
data = data.loc[:,train_columns]

column_list = ["bus_route_id","in_out","station_code","route_station_code"]

encoded_column_dict = dict()

for element in column_list:
  encoded = encoder.fit_transform(data[element])
  encoded_column_dict[element] = encoder.classes_

  data["encoded_"+element] = encoded

In [39]:
data.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,day,day_type,day_name,is_weekend,Fri,Mon,Sat,Sun,Thr,Tue,is_holiday,day_name_6~7_ride_sum,day_name_7~8_ride_sum,day_name_8~9_ride_sum,day_name_9~10_ride_sum,day_name_10~11_ride_sum,day_name_11~12_ride_sum,day_name_6~7_ride_mean,day_name_7~8_ride_mean,day_name_8~9_ride_mean,day_name_9~10_ride_mean,day_name_10~11_ride_mean,day_name_11~12_ride_mean,day_name_6~7_takeoff_sum,day_name_7~8_takeoff_sum,day_name_8~9_takeoff_sum,day_name_9~10_takeoff_sum,day_name_10~11_takeoff_sum,day_name_11~12_takeoff_sum,day_name_6~7_takeoff_mean,day_name_7~8_takeoff_mean,day_name_8~9_takeoff_mean,day_name_9~10_takeoff_mean,day_name_10~11_takeoff_mean,day_name_11~12_takeoff_mean,day_type_6~7_ride_sum,day_type_7~8_ride_sum,day_type_8~9_ride_sum,day_type_9~10_ride_sum,day_type_10~11_ride_sum,day_type_11~12_ride_sum,day_type_6~7_ride_mean,day_type_7~8_ride_mean,day_type_8~9_ride_mean,...,in_out_6~7_takeoff_sum,in_out_7~8_takeoff_sum,in_out_8~9_takeoff_sum,in_out_9~10_takeoff_sum,in_out_10~11_takeoff_sum,in_out_11~12_takeoff_sum,dis_jeju,dis_gosan,dis_seongsan,dis_po,dist_name,gosan,jeju,po,ride_sum,takeoff_sum,6~7_ride_date_sum,7~8_ride_date_sum,8~9_ride_date_sum,9~10_ride_date_sum,10~11_ride_date_sum,11~12_ride_date_sum,6~8r,8~10r,10~12r,6~8t,8~10t,10~12t,route_station_code,bus_route_id_mean,bus_route_id_sum,station_code_mean,station_code_sum,route_station_code_mean,route_station_code_sum,congestion,6~7_ride_mean,6~7_ride_sum,7~8_ride_mean,7~8_ride_sum,8~9_ride_mean,8~9_ride_sum,9~10_ride_mean,9~10_ride_sum,10~11_ride_mean,10~11_ride_sum,11~12_ride_mean,11~12_ride_sum,6~7_takeoff_mean,6~7_takeoff_sum,7~8_takeoff_mean,7~8_takeoff_sum,8~9_takeoff_mean,8~9_takeoff_sum,9~10_takeoff_mean,9~10_takeoff_sum,10~11_takeoff_mean,10~11_takeoff_sum,11~12_takeoff_mean,11~12_takeoff_sum,is_train,encoded_bus_route_id,encoded_in_out,encoded_station_code,encoded_route_station_code
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.3767,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.21764,0.383422,0.508273,...,1637.0,3672.0,5215.0,4366.0,3539.0,3285.0,4.286659,37.692766,37.713630,27.841879,jeju,0.0,1.0,0.0,16.0,0.0,2942.0,4518.0,5682.0,6275.0,6774.0,6731.0,1.0,7.0,8.0,0.0,0.0,0.0,4270000_344,3.104381,2409.0,1.466667,44.0,1.466667,44.0,5.0,1.633333,49.0,2.733333,82.0,2.733333,82.0,2.433333,73.0,2.333333,70.0,2.100000,63.0,0.000000,0.0,0.166667,5.0,0.000000,0.0,0.100000,3.0,0.133333,4.0,0.166667,5.0,True,0,1,322,31053
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.3767,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.21764,0.383422,0.508273,...,1637.0,3672.0,5215.0,4366.0,3539.0,3285.0,4.966848,37.008492,38.465822,27.996179,jeju,0.0,1.0,0.0,22.0,0.0,2942.0,4518.0,5682.0,6275.0,6774.0,6731.0,5.0,6.0,11.0,0.0,0.0,0.0,4270000_357,3.104381,2409.0,4.178218,6330.0,5.366667,161.0,5.0,0.900000,27.0,4.066667,122.0,4.133333,124.0,3.766667,113.0,5.466667,164.0,4.833333,145.0,0.000000,0.0,0.000000,0.0,0.200000,6.0,0.066667,2.0,0.033333,1.0,0.266667,8.0,True,0,1,335,31054
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988

In [40]:
encoded_column_dict

{'bus_route_id': array([ 4270000,  4280000,  7990000,  8170000,  8180000,  8880000,
        17010000, 20010000, 20020000, 20040000, 20050000, 21000000,
        21010000, 21020000, 21040000, 21070000, 21080000, 21090000,
        21100000, 21110000, 21120000, 21150000, 21160000, 21180000,
        21200000, 21220000, 21230000, 21270000, 21290000, 21300000,
        21310000, 21330000, 21340000, 21350000, 21360000, 21370000,
        21390000, 21400000, 21410000, 21420000, 21430000, 21440000,
        21450000, 21460000, 21470000, 21480000, 21500000, 21510000,
        21520000, 21540000, 21550000, 21570000, 21580000, 21590000,
        21600000, 21610000, 21620000, 21720000, 21780000, 21790000,
        21800000, 21810000, 21820000, 21830000, 21850000, 21860000,
        21880000, 21900000, 21910000, 21920000, 21930000, 21940000,
        21950000, 21970000, 21980000, 22180000, 22190000, 22200000,
        22210000, 22220000, 22230000, 22240000, 22250000, 22340000,
        22350000, 22360000, 2237

#### 버스카드 승하차 태그 데이터

##### 버스 승하차 태그시간 변환

In [0]:
def time_converter(str_time,is_geton=True):
  
  result = ""

  if is_geton:

    add_str = "_ride"

    if str_time >= '06:00:00' and str_time < '07:00:00':
      result = f"6~7{add_str}"
    elif str_time >= '07:00:00' and str_time < '08:00:00':
      result = f"7~8{add_str}"
    elif str_time >= '08:00:00' and str_time < '09:00:00':
      result = f"8~9{add_str}"
    elif str_time >= '09:00:00' and str_time < '10:00:00':
      result = f"9~10{add_str}"
    elif str_time >= '10:00:00' and str_time < '11:00:00':
      result = f"10~11{add_str}"
    elif str_time >= '11:00:00' and str_time < '12:00:00':
      result = f"11~12{add_str}"

  else:

    add_str = "_takeoff"

    if str_time is None:
      result = "NaN"
    elif str_time >= '06:00:00' and str_time < '07:00:00':
      result = f"6~7{add_str}"
    elif str_time >= '07:00:00' and str_time < '08:00:00':
      result = f"7~8{add_str}"
    elif str_time >= '08:00:00' and str_time < '09:00:00':
      result = f"8~9{add_str}"
    elif str_time >= '09:00:00' and str_time < '10:00:00':
      result = f"9~10{add_str}"
    elif str_time >= '10:00:00' and str_time < '11:00:00':
      result = f"10~11{add_str}"
    elif str_time >= '11:00:00' and str_time < '12:00:00':
      result = f"11~12{add_str}"
    elif str_time >= '12:00:00' and str_time < '13:00:00':
      result = f"12~13{add_str}"
    elif str_time >= '13:00:00' and str_time < '14:00:00':
      result = f"13~14{add_str}"
    elif str_time >= '14:00:00' and str_time < '15:00:00':
      result = f"14~15{add_str}"
    elif str_time >= '15:00:00' and str_time < '16:00:00':
      result = f"15~16{add_str}"

  return result

In [0]:
bts["timeline"] = bts.geton_time.apply(lambda x:time_converter(x))

##### 카테고리별 총 승차인원

In [43]:
dummy = bts.groupby(["bus_route_id","user_category"])["user_count"].sum().reset_index()
df_bus_user_cat = pd.pivot_table(data=dummy,index="bus_route_id",columns="user_category",values="user_count",aggfunc=np.max).fillna(0)
df_bus_user_cat.columns = list(map(lambda x:"user_cat_sum_"+str(x),df_bus_user_cat.columns))
data = pd.merge(left=data,right=df_bus_user_cat,how='left',on='bus_route_id')
data.loc[:,df_bus_user_cat.columns].fillna(0,inplace=True)
data

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,day,day_type,day_name,is_weekend,Fri,Mon,Sat,Sun,Thr,Tue,is_holiday,day_name_6~7_ride_sum,day_name_7~8_ride_sum,day_name_8~9_ride_sum,day_name_9~10_ride_sum,day_name_10~11_ride_sum,day_name_11~12_ride_sum,day_name_6~7_ride_mean,day_name_7~8_ride_mean,day_name_8~9_ride_mean,day_name_9~10_ride_mean,day_name_10~11_ride_mean,day_name_11~12_ride_mean,day_name_6~7_takeoff_sum,day_name_7~8_takeoff_sum,day_name_8~9_takeoff_sum,day_name_9~10_takeoff_sum,day_name_10~11_takeoff_sum,day_name_11~12_takeoff_sum,day_name_6~7_takeoff_mean,day_name_7~8_takeoff_mean,day_name_8~9_takeoff_mean,day_name_9~10_takeoff_mean,day_name_10~11_takeoff_mean,day_name_11~12_takeoff_mean,day_type_6~7_ride_sum,day_type_7~8_ride_sum,day_type_8~9_ride_sum,day_type_9~10_ride_sum,day_type_10~11_ride_sum,day_type_11~12_ride_sum,day_type_6~7_ride_mean,day_type_7~8_ride_mean,day_type_8~9_ride_mean,...,dis_seongsan,dis_po,dist_name,gosan,jeju,po,ride_sum,takeoff_sum,6~7_ride_date_sum,7~8_ride_date_sum,8~9_ride_date_sum,9~10_ride_date_sum,10~11_ride_date_sum,11~12_ride_date_sum,6~8r,8~10r,10~12r,6~8t,8~10t,10~12t,route_station_code,bus_route_id_mean,bus_route_id_sum,station_code_mean,station_code_sum,route_station_code_mean,route_station_code_sum,congestion,6~7_ride_mean,6~7_ride_sum,7~8_ride_mean,7~8_ride_sum,8~9_ride_mean,8~9_ride_sum,9~10_ride_mean,9~10_ride_sum,10~11_ride_mean,10~11_ride_sum,11~12_ride_mean,11~12_ride_sum,6~7_takeoff_mean,6~7_takeoff_sum,7~8_takeoff_mean,7~8_takeoff_sum,8~9_takeoff_mean,8~9_takeoff_sum,9~10_takeoff_mean,9~10_takeoff_sum,10~11_takeoff_mean,10~11_takeoff_sum,11~12_takeoff_mean,11~12_takeoff_sum,is_train,encoded_bus_route_id,encoded_in_out,encoded_station_code,encoded_route_station_code,user_cat_sum_1,user_cat_sum_2,user_cat_sum_4,user_cat_sum_6,user_cat_sum_27,user_cat_sum_28,user_cat_sum_29,user_cat_sum_30
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.376700,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.217640,0.383422,0.508273,...,37.713630,27.841879,jeju,0.0,1.0,0.0,16.0,0.0,2942.0,4518.0,5682.0,6275.0,6774.0,6731.0,1.0,7.0,8.0,0.0,0.0,0.0,4270000_344,3.104381,2409.0,1.466667,44.0,1.466667,44.0,5.0,1.633333,49.0,2.733333,82.0,2.733333,82.0,2.433333,73.0,2.333333,70.0,2.100000,63.0,0.000000,0.0,0.166667,5.0,0.000000,0.0,0.100000,3.0,0.133333,4.0,0.166667,5.0,True,0,1,322,31053,8158.0,72.0,38.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.376700,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.217640,0.383422,0.508273,...,38.465822,27.996179,jeju,0.0,1.0,0.0,22.0,0.0,2942.0,4518.0,5682.0,6275.0,6774.0,6731.0,5.0,6.0,11.0,0.0,0.0,0.0,4270000_357,3.104381,2409.0,4.178218,6330.0,5.366667,161.0,5.0,0.900000,27.0,4.066667,122.0,4.133333,124.0,3.766667,113.0,5.466667,164.0,4.833333,145.0,0.000000,0.0,0.000000,0.0,0.200000,6.0,0.066667,2.0,0.033333,1.0,0.266667,8.0,True,0,1,335,31054,8158.0,72.0,38.0,0.0,0.0,0.0,0.0,0.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,

##### 카테고리별 승객 비율

In [44]:
dummy = bts.groupby(["bus_route_id","user_category"])["user_count"].sum().reset_index()
dummy.columns = list(dummy.columns[:2]) + ["user_count_by_cat"]
dummy2 = bts.groupby(["bus_route_id"])["user_count"].sum().reset_index()
dummy2.columns = list(dummy2.columns[:1]) + ["user_count_total"]

df_user_cat_ratio = pd.merge(left=dummy,right=dummy2,how='inner',on='bus_route_id')
df_user_cat_ratio["user_cat_ratio"] = df_user_cat_ratio["user_count_by_cat"] / df_user_cat_ratio["user_count_total"]
df_user_cat_ratio = pd.pivot_table(data=df_user_cat_ratio,values="user_cat_ratio",index="bus_route_id",columns="user_category",aggfunc=np.max,fill_value=0)
df_user_cat_ratio.columns = list(map(lambda x:"user_cat_ratio_"+str(x),df_user_cat_ratio.columns))
data = pd.merge(left=data,right=df_user_cat_ratio,how='left',on='bus_route_id')
data.loc[:,df_user_cat_ratio.columns].fillna(0,inplace=True)
data

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,day,day_type,day_name,is_weekend,Fri,Mon,Sat,Sun,Thr,Tue,is_holiday,day_name_6~7_ride_sum,day_name_7~8_ride_sum,day_name_8~9_ride_sum,day_name_9~10_ride_sum,day_name_10~11_ride_sum,day_name_11~12_ride_sum,day_name_6~7_ride_mean,day_name_7~8_ride_mean,day_name_8~9_ride_mean,day_name_9~10_ride_mean,day_name_10~11_ride_mean,day_name_11~12_ride_mean,day_name_6~7_takeoff_sum,day_name_7~8_takeoff_sum,day_name_8~9_takeoff_sum,day_name_9~10_takeoff_sum,day_name_10~11_takeoff_sum,day_name_11~12_takeoff_sum,day_name_6~7_takeoff_mean,day_name_7~8_takeoff_mean,day_name_8~9_takeoff_mean,day_name_9~10_takeoff_mean,day_name_10~11_takeoff_mean,day_name_11~12_takeoff_mean,day_type_6~7_ride_sum,day_type_7~8_ride_sum,day_type_8~9_ride_sum,day_type_9~10_ride_sum,day_type_10~11_ride_sum,day_type_11~12_ride_sum,day_type_6~7_ride_mean,day_type_7~8_ride_mean,day_type_8~9_ride_mean,...,6~7_ride_date_sum,7~8_ride_date_sum,8~9_ride_date_sum,9~10_ride_date_sum,10~11_ride_date_sum,11~12_ride_date_sum,6~8r,8~10r,10~12r,6~8t,8~10t,10~12t,route_station_code,bus_route_id_mean,bus_route_id_sum,station_code_mean,station_code_sum,route_station_code_mean,route_station_code_sum,congestion,6~7_ride_mean,6~7_ride_sum,7~8_ride_mean,7~8_ride_sum,8~9_ride_mean,8~9_ride_sum,9~10_ride_mean,9~10_ride_sum,10~11_ride_mean,10~11_ride_sum,11~12_ride_mean,11~12_ride_sum,6~7_takeoff_mean,6~7_takeoff_sum,7~8_takeoff_mean,7~8_takeoff_sum,8~9_takeoff_mean,8~9_takeoff_sum,9~10_takeoff_mean,9~10_takeoff_sum,10~11_takeoff_mean,10~11_takeoff_sum,11~12_takeoff_mean,11~12_takeoff_sum,is_train,encoded_bus_route_id,encoded_in_out,encoded_station_code,encoded_route_station_code,user_cat_sum_1,user_cat_sum_2,user_cat_sum_4,user_cat_sum_6,user_cat_sum_27,user_cat_sum_28,user_cat_sum_29,user_cat_sum_30,user_cat_ratio_1,user_cat_ratio_2,user_cat_ratio_4,user_cat_ratio_6,user_cat_ratio_27,user_cat_ratio_28,user_cat_ratio_29,user_cat_ratio_30
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.376700,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.217640,0.383422,0.508273,...,2942.0,4518.0,5682.0,6275.0,6774.0,6731.0,1.0,7.0,8.0,0.0,0.0,0.0,4270000_344,3.104381,2409.0,1.466667,44.0,1.466667,44.0,5.0,1.633333,49.0,2.733333,82.0,2.733333,82.0,2.433333,73.0,2.333333,70.0,2.100000,63.0,0.000000,0.0,0.166667,5.0,0.000000,0.0,0.100000,3.0,0.133333,4.0,0.166667,5.0,True,0,1,322,31053,8158.0,72.0,38.0,0.0,0.0,0.0,0.0,0.0,0.986696,0.008708,0.004596,0.0,0.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.376700,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.217640,0.383422,0.508273,...,2942.0,4518.0,5682.0,6275.0,6774.0,6731.0,5.0,6.0,11.0,0.0,0.0,0.0,4270000_357,3.104381,2409.0,4.178218,6330.0,5.366667,161.0,5.0,0.900000,27.0,4.066667,122.0,4.133333,124.0,3.766667,113.0,5.466667,164.0,4.833333,145.0,0.000000,0.0,0.000000,0.0,0.200000,6.0,0.066667,2.0,0.033333,1.0,0.266667,8.0,True,0,1,335,31054,8158.0,72.0,38.0,0.0,0.0,0.0,0.0,0.0,0.986696,0.008708,0.004596,0.0,0.0,0.0,0.0,0.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,

##### 날짜별 특정 노선의 정류장을 지나간 배차된 차량의 수

In [0]:
list_geton = ['user_card_id', 'bus_route_id', 'vhc_id', 'geton_date', 'geton_time',
       'geton_station_code', 'geton_station_name','user_category', 'user_count']
list_getoff = ['user_card_id', 'bus_route_id', 'vhc_id', 'getoff_date',
       'getoff_time', 'getoff_station_code', 'getoff_station_name',
       'user_category', 'user_count']

df_bus_bts_geton = bts.loc[:,list_geton]
df_bus_bts_getoff = bts.loc[:,list_getoff]

In [0]:
list_df_bus_bts_geton_index_col = ["geton_date","bus_route_id","geton_station_code"]
list_df_bus_bts_getoff_index_col = ["getoff_date","bus_route_id","getoff_station_code"]

In [0]:
df_bus_bts_geton_passby = df_bus_bts_geton.groupby(list_df_bus_bts_geton_index_col)["vhc_id"].count().to_frame()
df_bus_bts_geton_passby.columns = ["geton_vhc_passby_count"]

df_bus_bts_getoff_passby = df_bus_bts_getoff.groupby(list_df_bus_bts_getoff_index_col)["vhc_id"].count().to_frame()
df_bus_bts_getoff_passby.columns = ["getoff_vhc_passby_count"]

In [0]:
df_bus_bts_geton_passby = df_bus_bts_geton_passby.reset_index()
df_bus_bts_getoff_passby = df_bus_bts_getoff_passby.reset_index()

df_bus_bts_geton_passby["geton_vhc_passby_count"].fillna(value=0,inplace=True)
df_bus_bts_getoff_passby["getoff_vhc_passby_count"].fillna(value=0,inplace=True)

In [49]:
data.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,day,day_type,day_name,is_weekend,Fri,Mon,Sat,Sun,Thr,Tue,is_holiday,day_name_6~7_ride_sum,day_name_7~8_ride_sum,day_name_8~9_ride_sum,day_name_9~10_ride_sum,day_name_10~11_ride_sum,day_name_11~12_ride_sum,day_name_6~7_ride_mean,day_name_7~8_ride_mean,day_name_8~9_ride_mean,day_name_9~10_ride_mean,day_name_10~11_ride_mean,day_name_11~12_ride_mean,day_name_6~7_takeoff_sum,day_name_7~8_takeoff_sum,day_name_8~9_takeoff_sum,day_name_9~10_takeoff_sum,day_name_10~11_takeoff_sum,day_name_11~12_takeoff_sum,day_name_6~7_takeoff_mean,day_name_7~8_takeoff_mean,day_name_8~9_takeoff_mean,day_name_9~10_takeoff_mean,day_name_10~11_takeoff_mean,day_name_11~12_takeoff_mean,day_type_6~7_ride_sum,day_type_7~8_ride_sum,day_type_8~9_ride_sum,day_type_9~10_ride_sum,day_type_10~11_ride_sum,day_type_11~12_ride_sum,day_type_6~7_ride_mean,day_type_7~8_ride_mean,day_type_8~9_ride_mean,...,6~7_ride_date_sum,7~8_ride_date_sum,8~9_ride_date_sum,9~10_ride_date_sum,10~11_ride_date_sum,11~12_ride_date_sum,6~8r,8~10r,10~12r,6~8t,8~10t,10~12t,route_station_code,bus_route_id_mean,bus_route_id_sum,station_code_mean,station_code_sum,route_station_code_mean,route_station_code_sum,congestion,6~7_ride_mean,6~7_ride_sum,7~8_ride_mean,7~8_ride_sum,8~9_ride_mean,8~9_ride_sum,9~10_ride_mean,9~10_ride_sum,10~11_ride_mean,10~11_ride_sum,11~12_ride_mean,11~12_ride_sum,6~7_takeoff_mean,6~7_takeoff_sum,7~8_takeoff_mean,7~8_takeoff_sum,8~9_takeoff_mean,8~9_takeoff_sum,9~10_takeoff_mean,9~10_takeoff_sum,10~11_takeoff_mean,10~11_takeoff_sum,11~12_takeoff_mean,11~12_takeoff_sum,is_train,encoded_bus_route_id,encoded_in_out,encoded_station_code,encoded_route_station_code,user_cat_sum_1,user_cat_sum_2,user_cat_sum_4,user_cat_sum_6,user_cat_sum_27,user_cat_sum_28,user_cat_sum_29,user_cat_sum_30,user_cat_ratio_1,user_cat_ratio_2,user_cat_ratio_4,user_cat_ratio_6,user_cat_ratio_27,user_cat_ratio_28,user_cat_ratio_29,user_cat_ratio_30
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.3767,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.21764,0.383422,0.508273,...,2942.0,4518.0,5682.0,6275.0,6774.0,6731.0,1.0,7.0,8.0,0.0,0.0,0.0,4270000_344,3.104381,2409.0,1.466667,44.0,1.466667,44.0,5.0,1.633333,49.0,2.733333,82.0,2.733333,82.0,2.433333,73.0,2.333333,70.0,2.100000,63.0,0.000000,0.0,0.166667,5.0,0.000000,0.0,0.100000,3.0,0.133333,4.0,0.166667,5.0,True,0,1,322,31053,8158.0,72.0,38.0,0.0,0.0,0.0,0.0,0.0,0.986696,0.008708,0.004596,0.0,0.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.3767,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.21764,0.383422,0.508273,...,2942.0,4518.0,5682.0,6275.0,6774.0,6731.0,5.0,6.0,11.0,0.0,0.0,0.0,4270000_357,3.104381,2409.0,4.178218,6330.0,5.366667,161.0,5.0,0.900000,27.0,4.066667,122.0,4.133333,124.0,3.766667,113.0,5.466667,164.0,4.833333,145.0,0.000000,0.0,0.000000,0.0,0.200000,6.0,0.066667,2.0,0.033333,1.0,0.266667,8.0,True,0,1,335,31054,8158.0,72.0,38.0,0.0,0.0,0.0,0.0,0.0,0.986696,0.008708,0.004596,0.0,0.0,0.0,0.0,0.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.2199

In [0]:
data = pd.merge(left=data,right=df_bus_bts_geton_passby,how='left',left_on=["date","bus_route_id","station_code"],right_on=list_df_bus_bts_geton_index_col)
data.drop(columns=["geton_date","geton_station_code"],axis=1,inplace=True)
data["geton_vhc_passby_count"] = data["geton_vhc_passby_count"].fillna(0)

data = pd.merge(left=data,right=df_bus_bts_getoff_passby,how='left',left_on=["date","bus_route_id","station_code"],right_on=list_df_bus_bts_getoff_index_col)
data.drop(columns=["getoff_date","getoff_station_code"],axis=1,inplace=True)
data["getoff_vhc_passby_count"] = data["getoff_vhc_passby_count"].fillna(0)

### 날씨 데이터

##### 날씨 데이터 병합

In [0]:
weather_data.index = pd.date_range(start="2019-09-01",end="2019-10-16")
weather_data = weather_data.reset_index()
weather_data.columns = ["date"] + list(weather_data.columns[1:])
weather_data["date"]  = weather_data["date"].astype("str")
data = pd.merge(left=data,right=weather_data,how="left",on="date")

##### 강수 여부

In [0]:
data["is_rain"] = data["일강수_10"].apply(lambda x: False if x == 0 else True)

In [0]:
encoder = OneHotEncoder()
encoded_weather = encoder.fit_transform(data["현재일기_10"].values.reshape(-1,1))
dummy = pd.DataFrame(encoded_weather.todense(),columns=map(lambda x:x[3:] ,encoder.get_feature_names()))
dummy = dummy.loc[:,dummy.columns[:-1]]
data = pd.concat([data,dummy],axis=1)

In [54]:
data

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,day,day_type,day_name,is_weekend,Fri,Mon,Sat,Sun,Thr,Tue,is_holiday,day_name_6~7_ride_sum,day_name_7~8_ride_sum,day_name_8~9_ride_sum,day_name_9~10_ride_sum,day_name_10~11_ride_sum,day_name_11~12_ride_sum,day_name_6~7_ride_mean,day_name_7~8_ride_mean,day_name_8~9_ride_mean,day_name_9~10_ride_mean,day_name_10~11_ride_mean,day_name_11~12_ride_mean,day_name_6~7_takeoff_sum,day_name_7~8_takeoff_sum,day_name_8~9_takeoff_sum,day_name_9~10_takeoff_sum,day_name_10~11_takeoff_sum,day_name_11~12_takeoff_sum,day_name_6~7_takeoff_mean,day_name_7~8_takeoff_mean,day_name_8~9_takeoff_mean,day_name_9~10_takeoff_mean,day_name_10~11_takeoff_mean,day_name_11~12_takeoff_mean,day_type_6~7_ride_sum,day_type_7~8_ride_sum,day_type_8~9_ride_sum,day_type_9~10_ride_sum,day_type_10~11_ride_sum,day_type_11~12_ride_sum,day_type_6~7_ride_mean,day_type_7~8_ride_mean,day_type_8~9_ride_mean,...,station_code_mean,station_code_sum,route_station_code_mean,route_station_code_sum,congestion,6~7_ride_mean,6~7_ride_sum,7~8_ride_mean,7~8_ride_sum,8~9_ride_mean,8~9_ride_sum,9~10_ride_mean,9~10_ride_sum,10~11_ride_mean,10~11_ride_sum,11~12_ride_mean,11~12_ride_sum,6~7_takeoff_mean,6~7_takeoff_sum,7~8_takeoff_mean,7~8_takeoff_sum,8~9_takeoff_mean,8~9_takeoff_sum,9~10_takeoff_mean,9~10_takeoff_sum,10~11_takeoff_mean,10~11_takeoff_sum,11~12_takeoff_mean,11~12_takeoff_sum,is_train,encoded_bus_route_id,encoded_in_out,encoded_station_code,encoded_route_station_code,user_cat_sum_1,user_cat_sum_2,user_cat_sum_4,user_cat_sum_6,user_cat_sum_27,user_cat_sum_28,user_cat_sum_29,user_cat_sum_30,user_cat_ratio_1,user_cat_ratio_2,user_cat_ratio_4,user_cat_ratio_6,user_cat_ratio_27,user_cat_ratio_28,user_cat_ratio_29,user_cat_ratio_30,geton_vhc_passby_count,getoff_vhc_passby_count,현재일기_10,현재기온_10,일강수_10,is_rain,구름많음,구름조금,맑음,박무,보통비계속,비 끝남,소나기 끝,약한비계속,약한비단속
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.376700,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.217640,0.383422,0.508273,...,1.466667,44.0,1.466667,44.0,5.0,1.633333,49.0,2.733333,82.0,2.733333,82.0,2.433333,73.0,2.333333,70.0,2.100000,63.0,0.000000,0.0,0.166667,5.0,0.000000,0.0,0.100000,3.0,0.133333,4.0,0.166667,5.0,True,0,1,322,31053,8158.0,72.0,38.0,0.0,0.0,0.0,0.0,0.0,0.986696,0.008708,0.004596,0.0,0.0,0.0,0.0,0.0,16.0,0.0,흐림,25.300,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.367729,0.376700,21349.0,37611.0,49858.0,54139.0,55810.0,56900.0,0.217640,0.383422,0.508273,...,4.178218,6330.0,5.366667,161.0,5.0,0.900000,27.0,4.066667,122.0,4.133333,124.0,3.766667,113.0,5.466667,164.0,4.833333,145.0,0.000000,0.0,0.000000,0.0,0.200000,6.0,0.066667,2.0,0.033333,1.0,0.266667,8.0,True,0,1,335,31054,8158.0,72.0,38.0,0.0,0.0,0.0,0.0,0.0,0.986696,0.008708,0.004596,0.0,0.0,0.0,0.0,0.0,22.0,0.0,흐림,25.300,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,01,weekend,Sun,True,0.0,0.0,0.0,1.0,0.0,0.0,False,11548.0,18916.0,25386.0,27881.0,29205.0,28988.0,0.219937,0.360264,0.483488,0.531006,0.556222,0.552089,5156.0,11011.0,15799.0,18138.0,19308.0,19779.0,0.098198,0.209709,0.300899,0.345446,0.